In [1]:
import torch
import pyvista

from torchfem import Solid
from torchfem.mesh import cube_hexa
from torchfem.materials import IsotropicElasticity3D, IsotropicKirchhoff3D
from torchfem.io import export_mesh

# Set default data type to double precision
torch.set_default_dtype(torch.float64)

# Generate cube
L = 5.0
P = -5
nodes, elements = cube_hexa(25, 5, 5, L, 1.0, 1.0)
tip = (nodes[:, 0] == L) & (nodes[:, 1] == 0.5) & (nodes[:, 2] == 0.5)

In [2]:
# Create model with small strain isotropic elasticity
material = IsotropicElasticity3D(E=1000.0, nu=0.3)
beam1 = Solid(nodes, elements, material)

# Assign boundary conditions
beam1.constraints[nodes[:, 0] == 0, :] = True
beam1.forces[tip, 2] = P

# Solve
u1, f1, σ1, F1, α1 = beam1.solve()

# Compute strains
ε1 = 0.5 * (F1.transpose(-1, -2) + F1) - torch.eye(3)

print(f"Tip deflection: {u1[tip]}")
print(f"Maximum stress σ_xx: {σ1.max():.3f}")

# Plot
beam1.plot(u=u1, element_property={"Stress": σ1[:, 0]})

Tip deflection: tensor([[-3.4840e-11,  1.6717e-10, -2.4886e+00]])
Maximum stress σ_xx: 110.315


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [3]:
# Create model with Kirchhoff material
material = IsotropicKirchhoff3D(E=1000.0, nu=0.3)
beam2 = Solid(nodes, elements, material)

# Assign boundary conditions
beam2.constraints[nodes[:, 0] == 0, :] = True
beam2.forces[tip, 2] = P

# Solve
u2, f2, σ2, F2, α2 = beam2.solve(max_iter=100)

# Compute material rotation from polar decomposition of deformation gradient
U, _, Vh = torch.linalg.svd(F2)
R = U @ Vh

print(f"Tip deflection: {u2[tip]}")
print(f"Maximum stress σ_xx: {σ2.max():.3f}")

# Plot
beam2.plot(u=u2, element_property={"Stress": σ2[:, 0]}, orientations=R, opacity=0.5)
export_mesh(beam2, "temp.vtu", nodal_data={"U2": u2, "U1": u1})

Tip deflection: tensor([[-7.7360e-01,  1.2317e-16, -2.3706e+00]])
Maximum stress σ_xx: 120.508


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…